In [7]:
import linguamind.linalg as la
import linguamind.nn as nn
import linguamind.nlp as nlp

from collections import Counter
import random

path ='/Users/amberedmundson/Laboratory/datasets/harryPotter/harryPotterAsciiCleaned.txt'
# path = '/Users/amberedmundson/Laboratory/clones/word2vec/text.txt'

v = nlp.Vocab()
t = nlp.Text(path,v)

v2 = nlp.Vocab(t.vocab)
v2.sort(0)
t.ChangeVocab(v2)

v2.InitUnigramTable()

vocab_size = v2.size
dim = 10

seed = la.Seed(1)

syn0 = nn.SparseLinearInput(vocab_size,dim)
syn0.weights.uniform(seed)
syn0.weights -= 0.5
syn0.weights /= dim

syn1neg = nn.SparseLinearOutput(dim,vocab_size)
syn1neg.weights.zero()

<linguamind.linalg.Matrix; proxy of <Swig Object of type 'Matrix *' at 0x104a8f960> >

In [8]:
layers = list()
layers.append(syn0)
layers.append(syn1neg)
layers.append(nn.Sigmoid(vocab_size))

mlp = nn.Sequential(layers)
criterion = nn.MSECriterion()
optim = nn.StochasticGradient(mlp,criterion)

v2.getUnigramValue(45000000)
cbow = nn.CBOW(t.sentences,v2,5,5)

In [9]:
mlp.forward(cbow.getInputIndicesReference(),cbow.getOutputIndicesReference())

<linguamind.linalg.Vector; proxy of <Swig Object of type 'Vector *' at 0x105c1e030> >

In [10]:
syn0.getOutput().get()

(-0.05706481635570526,
 -0.09654388576745987,
 0.07289429008960724,
 -0.03881378099322319,
 0.01391296274960041,
 -0.08809050917625427,
 0.02868652530014515,
 0.08941497653722763,
 0.00022277794778347015,
 0.04155426472425461)

In [62]:
%%timeit
optim.train(cbow,0.01,1)

1 loop, best of 3: 1.44 s per loop


In [63]:
word2vec = {}
vocab = list()
for i in range(v2.size):
    vocab.append(v2.getTermAtIndex(i).letters)
    word2vec[v2.getTermAtIndex(i).letters] = syn0.weights[i]

In [64]:
sim = Counter()
for word in vocab:
    sim[word] = word2vec['hermione'].dot(word2vec[word])
sim.most_common()

[('biscuit', 0.0030132243409752846),
 ('auror', 0.0030132243409752846),
 ('depth', 0.0030132243409752846),
 ('sectum', 0.0030132243409752846),
 ('clearer:', 0.0030132243409752846),
 ('candle', 0.0030132243409752846),
 ('declined', 0.0030132243409752846),
 ('griphook-free', 0.0030132243409752846),
 ('perching', 0.0030132243409752846),
 ('din', 0.0030132243409752846),
 ('youngsters', 0.0030132243409752846),
 ('intruder?', 0.0030132243409752846),
 ('hermione', 0.0030132243409752846),
 ('half-completed', 0.0030132243409752846),
 ('ricocheted', 0.0030132243409752846),
 ('dawlish', 0.0017603269079700112),
 ('sketch', 0.0017603269079700112),
 ('inwardly', 0.0017603269079700112),
 ('recovered', 0.0017603269079700112),
 ('periwinkle', 0.0017603269079700112),
 ('smethley', 0.0017603269079700112),
 ('form?', 0.0017603269079700112),
 ('evoked', 0.0017603269079700112),
 ('aura', 0.0017603269079700112),
 ('rest', 0.0017603269079700112),
 ('also!', 0.0017603269079700112),
 ('suspiciously', 0.00176032

In [53]:
sim = Counter()
for word in vocab:
    sim[word] = word2vec['hermione'].dot(word2vec[word])
sim.most_common()

[('hermione', 0.003181311534717679),
 ('notes;', 0.002070854650810361),
 ('windblown', 0.0020572540815919638),
 ('prototype', 0.0019918906036764383),
 ('repaid', 0.0019425066420808434),
 ('wandmaker?', 0.0018963883630931377),
 ('boost', 0.001891178428195417),
 ('audacious', 0.0018463338492438197),
 ('heather-covered', 0.001840349636040628),
 ('warning;', 0.0018306660931557417),
 ('knowledgeably', 0.001789494650438428),
 ('dash', 0.0017880459781736135),
 ('ooooh', 0.0017599838320165873),
 ('faces:', 0.0017543176654726267),
 ('freed?', 0.0017127224709838629),
 ('curvy', 0.0017085174331441522),
 ('columns', 0.0016870005056262016),
 ('bandy-legged', 0.001683913404121995),
 ('disbelievingly', 0.001681084162555635),
 ('(so', 0.0016679789405316114),
 ('warning', 0.0016588489525020123),
 ("'popkin", 0.0016579327639192343),
 ('discuss', 0.0016389498487114906),
 ('theater', 0.001629477133974433),
 ('belt', 0.0016280398704111576),
 ('deliberate:', 0.001625322736799717),
 ('attire', 0.001621545874

# Forward Prop Modification for Windowing Layers

after you have confirmed that you are doing everything that word2vec does (exactly the same!!!)... such that you duplicate results, you should try this highly experimental thing

For CBOW, you should be able to tweak the output embedding based on how the inputs change and the previous gradient instead of regenerating it every time. If your window size is 5, this should give you close to a 5 * 2 = 10x speedup for the input window generation (which is where most of the parameters are going anyway)

# Windowing Object Passed to SGD

It should basically have a "get next training example" method.

# Negative Sampling Layers

### Full

In [ ]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

In [ ]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

# Optim 1: From training examples to windows

### None... just the loop

Note... previously 100

In [ ]:
new_limit = int((100000/float(176216)) * 10000)

In [ ]:
new_limit

### None... just the loop

In [ ]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

In [ ]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

### Forward Prop Only

In [ ]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

In [ ]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

### Forward and Back Prop

In [ ]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

In [ ]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

### Full

In [ ]:
%%timeit
optim.train(sentences_as_indices[0:new_limit],target,alpha=0.01,iterations=1)

In [ ]:
%%timeit
optim.train(sentences_as_indices,target,alpha=0.01,iterations=1)

# Starting Point

### None... just the loop

In [ ]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

### Forward Prop Only

In [ ]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

# Forward and Back Prop

In [ ]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

### Total Again

In [ ]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

### Total

In [ ]:
%%timeit
optim.train(training_examples_context[0:100000],training_examples_preds[0:100000],target,alpha=0.01,iterations=1)

In [ ]:
%%timeit
for input_indices,output_indices in training_examples[0:20000]:
    error = (optim.train(input_indices,output_indices,target,alpha=0.01,iterations=1))